In [2]:
from bs4 import BeautifulSoup
import requests

def ingredients_from_all(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    ing_list = soup.find(id="mntl-structured-ingredients_1-0")
    ingredients = ing_list.find_all(class_="mntl-structured-ingredients__list")
    ig_t = ingredients[0].get_text()
    sa = ig_t.split('\n')
    i_list = [x for x in sa if len(x)>1]
    return i_list

In [3]:
# read and parse data from webscraping
url = 'https://www.cia.gov/the-world-factbook/field/electricity/'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')
country_names = soup.find_all(class_="h3")


In [4]:
c_list = soup.find_all('ul')[1] # getting country data lists
results = c_list.find_all('li')

countries = []
consumptions = []

for r in results:
    c = r.find(class_="h3").get_text()
    data = r.find('strong')
    d1 = data.next_sibling
    d2 = d1.next_sibling.next_sibling.next_sibling.next_sibling
    d3 = d2.next_sibling.next_sibling.next_sibling.next_sibling
    countries.append(c)
    consumptions.append(d2.get_text().split(' '))
kwh = [c[0] for c in consumptions]


In [5]:
import pandas as pd
df = pd.DataFrame({'Country': countries, 'Consumption': kwh})

import pycountry

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3

cc_col = []
for i, row in df.iterrows():
    cc_col.append(countries.get(row['Country'], 'Unknown code'))

df['code'] = cc_col

In [9]:
import plotly.express as px
df["Consumption"] = df["Consumption"].str.replace(',','')
df["Consumption"] = df["Consumption"].astype(float)
fig = px.choropleth(df, locations="code",
                    range_color=(0, max(df['Consumption'])),
                    color="Consumption", # lifeExp is a column of gapminder
                    hover_name="Country", # column to add to hover information
)
fig.show()